In [2]:
%load_ext autoreload
%autoreload

import io
import os
import sys
import django
import glob
import json
import re
import requests
from IPython.core.debugger import set_trace
from pathlib import Path
from django.core.files.base import ContentFile
from rest_framework import serializers
from tqdm.notebook import tqdm
from google.cloud import vision
from google.cloud import storage
from PIL import Image, ImageSequence
from gensim.models import Doc2Vec
from IPython.core.debugger import set_trace
import IPython
import collections
from googletrans import Translator
from google.cloud import translate_v2 as translate

parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)
django.setup()

import getch.models as m
from django.db.models import Q
from django.apps import apps
from django_currentuser.middleware import get_current_user

In [3]:
model_path = 'doc2vec.model'
d2v = Doc2Vec.load(model_path)

# 투표했을때 태그 저장하기

In [42]:
boo = m.Boo.objects.get(pk=200)

In [43]:
_tags_pos = collections.Counter(m.Pix.objects.first().tokens.split())
_tags_neg = collections.Counter(m.Pix.objects.last().tokens.split())

In [44]:
boo.postags = collections.Counter(boo.postags) + _tags_pos
boo.negtags = collections.Counter(boo.negtags) + _tags_neg

In [45]:
boo.save()

# 옷장에 넣을때 태그 저장하기
## 단일사용자 옷장에 있는 태그들 저장하기

In [5]:
boo = m.Boo.objects.get(pk=200)
_tokens = m.Pix.objects.filter(pick__collection__owner=boo.id).values_list('tokens', flat=True)
boo.coltags = collections.Counter(' '.join(_tokens).split())

In [6]:
boo.save()

## 모든사용자 옷장에 있는 태그들 저장하기

In [3]:
m.Boo.objects.count()

436

In [24]:
for b in tqdm(m.Boo.objects.all()):
    _tokens = m.Pix.objects.filter(pick__collection__owner=b.id).values_list('tokens', flat=True)
    b.coltags = collections.Counter(' '.join(_tokens).split())
    b.save()

  0%|          | 0/436 [00:00<?, ?it/s]

# 정밀프로파일 만들기

## 태그구성
* 최종태그 = 옷장 태그 + 선택한 이미지 태그 - 선택안한 이미지 태그
* 가장 흔한 태그 100개만으로 구성한다

In [12]:
n = 100
boo = m.Boo.objects.get(pk=200)
_coltags = dict(collections.Counter(boo.coltags).most_common(n)) #.elements()
_postags = dict(collections.Counter(boo.postags).most_common(n)) #.elements()
_negtags = dict(collections.Counter(boo.negtags).most_common(n)) #.elements()
col_vec = d2v.infer_vector(_coltags, epochs=500)
pos_vec = d2v.infer_vector(_postags, epochs=500)
neg_vec = d2v.infer_vector(_negtags, epochs=500)

In [280]:
# n = 100
# boo = m.Boo.objects.get(pk=200)
# _coltags = dict(collections.Counter(boo.coltags).most_common(n))
# _postags = dict(collections.Counter(boo.postags).most_common(n))
# _negtags = dict(collections.Counter(boo.negtags).most_common(n))
# col_vec = d2v.infer_vector(collections.Counter(_coltags).elements(), epochs=500)
# pos_vec = d2v.infer_vector(collections.Counter(_postags).elements(), epochs=500)
# neg_vec = d2v.infer_vector(collections.Counter(_negtags).elements(), epochs=500)

In [4]:
d2v.wv.most_similar(positive=[col_vec, pos_vec], negative=[neg_vec], topn=10)

[('아트인지', 0.6911892890930176),
 ('자료', 0.6499561071395874),
 ('boho', 0.6444910764694214),
 ('수채화', 0.6124486327171326),
 ('도매상담', 0.611363410949707),
 ('textile', 0.6112388968467712),
 ('미싱', 0.6105998754501343),
 ('design', 0.6089140772819519),
 ('jaybian', 0.6057155728340149),
 ('패션스카우트', 0.605593204498291)]

In [16]:
d2v.docvecs.most_similar(positive=[col_vec, pos_vec], negative=[neg_vec], topn=10)

[('aldo', 0.5479872822761536),
 ('maje', 0.5365695953369141),
 ('sandro', 0.5289338827133179),
 ('tomboy', 0.5181444883346558),
 ('veja', 0.5149912238121033),
 ('cherryla', 0.49839848279953003),
 ('daks', 0.49533137679100037),
 ('cutlerandgross', 0.4900127947330475),
 ('tomsshoes', 0.48941609263420105),
 ('needles', 0.48370176553726196)]

## 아이덴티티 할당

### 하드코딩하는 경우 (identity)

In [147]:
identity = {
    '럭셔리': '럭셔리 고급 호화 과시 명품 luxury 비싼 고가 expensive pricy pricey', 
    '캐주얼': '캐주얼 캐쥬얼 casual 스타일리시 스타일리쉬 stylish', 
    
    '유니크': '유니크 독특 독창 unique 개성 only 참신 신선 특이 아이디어 철학',
    '대중성': '대중 popular 널리 흔한 massive mass 대중성', 
    
    '정통성': '정통 클래식 classic 품격 약속 신뢰 믿음 예측 견고 품질 안정 트레디셔널', 
    '트렌디': '트렌디 트랜디 트렌드 트랜드 유행 trend trendy 변화 새로운 민감 예민 신상 최신',
    
    '포멀': '포멀 formal 노멀 normal 평범 일상 무난 기본 베이스 베이직 base basic',
    '액티브': '화제 인기 hot 튀는 액티브 active 앞서가는 실험 과감 선도 선구 대담', 
}

In [44]:
_vec = col_vec + pos_vec - neg_vec
sim = lambda v: d2v.wv.cosine_similarities(_vec, [d2v.infer_vector(v.split(), epochs=500)])[0]
_id = {k:sim(v) for k,v in identity.items()}; _id

{'럭셔리': 0.31003493,
 '캐주얼': 0.47410923,
 '유니크': 0.5219978,
 '대중성': 0.35203347,
 '정통성': 0.46993917,
 '트렌디': 0.44850352,
 '포멀': 0.5024961,
 '액티브': 0.5547902}

### 아이덴티티 정보를 DB에서 가져오는 경우

In [9]:
identities = m.Identity.objects.all()

In [14]:
_vec = pos_vec - neg_vec + col_vec
sim = lambda v: d2v.wv.cosine_similarities(_vec, [d2v.infer_vector(v.split(), epochs=500)])[0]
_id = {__id.name:sim(__id.keywords) for __id in identities}; _id

{'럭셔리': 0.33093706,
 '캐주얼': 0.5378571,
 '유니크': 0.5454433,
 '대중성': 0.40401742,
 '정통성': 0.5830707,
 '트렌디': 0.451235,
 '포멀': 0.3879838,
 '액티브': 0.43000537}

In [15]:
_max = max(_id.values())
_min = min(_id.values())
scaled = lambda x: round((x-_min)/(_max-_min) * 100)
{k:scaled(v) for k,v in _id.items()}

{'럭셔리': 0,
 '캐주얼': 82,
 '유니크': 85,
 '대중성': 29,
 '정통성': 100,
 '트렌디': 48,
 '포멀': 23,
 '액티브': 39}

In [4]:
d2v.estimate_memory()

{'doctag_lookup': 62800,
 'doctag_syn0': 125600,
 'vocab': 22986500,
 'vectors': 18389200,
 'syn1neg': 18389200,
 'total': 59953300}

In [3]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_doc2vec_format(model_path, limit=1000)

AttributeError: type object 'Word2VecKeyedVectors' has no attribute 'load_doc2vec_format'

In [5]:
from django.core.files.storage import default_storage

In [6]:
'sideb-proejct.appspot.com/model/doc2vec.model'

['/Users/quantlab/Documents/GitHub/maskplay/data',
 '/opt/anaconda3/lib/python38.zip',
 '/opt/anaconda3/lib/python3.8',
 '/opt/anaconda3/lib/python3.8/lib-dynload',
 '',
 '/Users/quantlab/Documents/GitHub/maskplay/venv/lib/python3.8/site-packages',
 '/Users/quantlab/Documents/GitHub/maskplay/venv/lib/python3.8/site-packages/IPython/extensions',
 '/Users/quantlab/.ipython',
 '/Users/quantlab/Documents/GitHub/maskplay']

In [10]:
d2v = Doc2Vec.load('gs://sideb-proejct.appspot.com/model/doc2vec.model')

AttributeError: 'Blob' object has no attribute 'download_as_bytes'

In [11]:
default_storage.exists('model/doc2vec.model')

True

In [12]:
file = default_storage.open('model/doc2vec.model', 'r')

In [18]:
file

<GoogleCloudFile: model/doc2vec.model>

In [4]:
model_path = 'doc2vec.model'
d2v = Doc2Vec.load(model_path)

In [5]:
d2v.train(vector_size=100, window=5, min_count=10, workers=4)

TypeError: train() got an unexpected keyword argument 'vector_size'

In [6]:
d2v.vector_size

100